In [56]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [57]:
def free_cash_rate(stock_id, year):
    
    debtinfo_url = f'https://goodinfo.tw/tw/StockFinDetail.asp?RPT_CAT=BS_M_YEAR&STOCK_ID={stock_id}'
    cashinfo_url = f'https://goodinfo.tw/tw/StockFinDetail.asp?RPT_CAT=CF_M_YEAR&STOCK_ID={stock_id}'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
        'Cookie': 'IS_TOUCH_DEVICE=F; _ga=GA1.1.181317566.1718532079; CLIENT%5FID=20240616180126875%5F36%2E229%2E52%2E63; TW_STOCK_BROWSE_LIST=2330; SCREEN_SIZE=WIDTH=2048&HEIGHT=1280; _ga_0LP5MLQS7E=GS1.1.1718532079.1.1.1718533794.55.0.0; FCNEC=%5B%5B%22AKsRol9SI_2fTGHpemG9YsrULojLbIxof0jMpZlU9D2QhtEJ1tcsb7DWXrxdOxPSr3M34xBLTu5R1-X5Y5YFzqjc7X5yv7XuyUZOC5efVbCaLev18nmzd81fN_QEOIPMrcGqwcyKtTt2dh-E6WHKVn-mBCwQQatztA%3D%3D%22%5D%5D'
    }

    debts = np.zeros(year)
    equity = np.zeros(year)
    bucash = np.zeros(year)
    invest_cash = np.zeros(year)
    free_cash = np.zeros(year)
    
    ## Part 1: 負債總額、股東權益總額
    res = requests.get(debtinfo_url, headers = headers)
    res.encoding = 'utf-8'
    # print(f"ddd={res.text}")
    soup = BeautifulSoup(res.text, 'html.parser')

    debt_tag = soup.find(string="負債總額")#, class_='bg_h1 fw_bold')
    for yyy in range(year):
        # print(f"debts_tag={debt_tag}")
        debt_tag = debt_tag.find_next('td')
        debts[yyy] = float(debt_tag.string.replace(',', ''))
        debt_tag = debt_tag.find_next('td')
    print(f"debts={debts}")
    
    
    soup = BeautifulSoup(res.text, 'html.parser')
    equity_tag = soup.find(string="股東權益總額")
    for yyy in range(year):
        # print(f"equity_tag={equity_tag}")
        equity_tag = equity_tag.find_next('td')
        equity[yyy] = float(equity_tag.string.replace(',', ''))
        equity_tag = equity_tag.find_next('td')
    print(f"equity={equity}")
    
    ## Part 2: 自由現金流 = 營業現金流 + 投資現金流
    # 營業現金流
    res = requests.post(cashinfo_url, headers = headers)

    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    bucash_tag = soup.find(string="營業活動之淨現金流入(出)")
    for yyy in range(year):
        bucash_tag = bucash_tag.find_next('td')
        bucash[yyy] = float(bucash_tag.string.replace(',', ''))
        # bucash_str = bucash_tag.find_next('td')
    print(f"bucash={bucash}")
    
    # 投資現金流
    invest_cash_tag = soup.find(string="投資活動之淨現金流入(出)")
    for yyy in range(year):
        invest_cash_tag = invest_cash_tag.find_next('td')
        invest_cash[yyy] = float(invest_cash_tag.string.replace(',', ''))
        # invest_cash_tag = invest_cash_tag.find_next('td')
    print(f"invest_cash={invest_cash}")
    
    # 自由現金流 = 營業現金流 + 投資現金流
    free_cash = bucash + invest_cash
    print(f"free_cash = {free_cash}")
    return free_cash/(debts+equity)

In [58]:
tsmc_fcr = free_cash_rate(2330, 3)
print(f"tsmc 最近3年之自由現金流報酬率={tsmc_fcr}")

debts=[23684. 20491. 20043.]
equity=[43236. 34833. 29605.]
bucash=[18262. 12420. 16106.]
invest_cash=[ -8648.  -9061. -11909.]
free_cash = [9614. 3359. 4197.]
tsmc 最近3年之自由現金流報酬率=[0.14366408 0.06071506 0.08453513]


In [59]:
url = "https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E5%85%AC%E5%8F%B8%E7%B8%BD%E5%B8%82%E5%80%BC%E6%9C%80%E9%AB%98%40%40%E5%85%AC%E5%8F%B8%E7%B8%BD%E5%B8%82%E5%80%BC%40%40%E5%85%AC%E5%8F%B8%E7%B8%BD%E5%B8%82%E5%80%BC%E6%9C%80%E9%AB%98&SHEET=%E5%85%AC%E5%8F%B8%E5%9F%BA%E6%9C%AC%E8%B3%87%E6%96%99"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'Cookie': 'IS_TOUCH_DEVICE=F; _ga=GA1.1.181317566.1718532079; CLIENT%5FID=20240616180126875%5F36%2E229%2E52%2E63; TW_STOCK_BROWSE_LIST=2330; SCREEN_SIZE=WIDTH=2048&HEIGHT=1280; _ga_0LP5MLQS7E=GS1.1.1718532079.1.1.1718533794.55.0.0; FCNEC=%5B%5B%22AKsRol9SI_2fTGHpemG9YsrULojLbIxof0jMpZlU9D2QhtEJ1tcsb7DWXrxdOxPSr3M34xBLTu5R1-X5Y5YFzqjc7X5yv7XuyUZOC5efVbCaLev18nmzd81fN_QEOIPMrcGqwcyKtTt2dh-E6WHKVn-mBCwQQatztA%3D%3D%22%5D%5D'
}

res = requests.get(url, headers = headers)
res.encoding = 'utf-8'

soup = BeautifulSoup(res.text, 'lxml')
# data = soup.select_one('#txtFinBody')
# data
nnn = soup.select_one('#divStockList')
dfs = pd.read_html(nnn.prettify())


C:\Users\Sasha\AppData\Local\Temp\ipykernel_3344\522205841.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(nnn.prettify())


In [60]:
dfs[0]

,排 名,代號,名稱,市 場,股價 日期,成交,漲跌 價,漲跌 幅,面值 (元),股本 (億),...,掛牌 年數,股票 期貨,選擇 權,權證,公司 債,私募 股,特別 股,產業別,董事長,總經理
0,1,2330,台積電,市,03/31,910,-42,-4.41,10,2593,...,30.6,有,有,有,有,NaN,NaN,半導體業,魏哲家,總裁: 魏哲家
1,2,2454,聯發科,市,03/31,1390,-75,-5.12,10,160,...,23.7,有,有,有,NaN,NaN,NaN,半導體業,蔡明介,陳冠州
2,3,2317,鴻海,市,03/31,146,-8,-5.19,10,1389,...,33.8,有,有,有,有,NaN,NaN,其他電子業,劉揚偉,劉揚偉
3,4,2881,富邦金,市,03/31,85.3,-2.9,-3.29,10,1527,...,23.3,有,有,有,有,NaN,有,金控業,蔡明興,韓蔚廷
4,5,2412,中華電,市,03/31,128.5,-1.5,-1.15,10,776,...,24.4,有,有,有,NaN,NaN,NaN,通信網路業,簡志誠,林榮賜
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,296,6442,光聖,市,03/31,412.5,-31.5,-7.09,10,7.6,...,9.72,NaN,NaN,NaN,NaN,NaN,NaN,通信網路業,"CHEN, STEVE",張英華
312,297,1560,中砂,市,03/31,215.5,-23,-9.64,10,14.6,...,20.2,NaN,NaN,有,NaN,NaN,NaN,電機機械,林伯全,謝榮哲
313,298,4743,合一,櫃,03/31,64.7,-5.3,-7.57,10,47.9,...,13.5,有,NaN,NaN,NaN,NaN,NaN,生技醫療業,郭憲壽,鄭淑玲
314,299,6278,台表科,市,03/31,106,-3,-2.75,10,29.2,...,21.1,有,NaN,有,NaN,NaN,NaN,光電業,伍開雲,伍允中
